In [5]:
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import numpy as np
import gensim


model = gensim.models.KeyedVectors.load_word2vec_format('ubercorpus.lowercased.tokenized.word2vec.300d.bz2', binary=False)


In [1]:
#https://github.com/EmilStenstrom/conllu
#https://github.com/UniversalDependencies/UD_English-EWT
from conllu import parse, parse_tree
from collections import OrderedDict
import numpy as np

In [2]:
with open('../UD_Ukrainian-IU/uk_iu-ud-train.conllu') as f:
    c= f.read()
    trees= parse(c)

In [3]:
tree=trees[0]

In [117]:
tree[5]

OrderedDict([('id', 6),
             ('form', 'була'),
             ('lemma', 'бути'),
             ('upostag', 'VERB'),
             ('xpostag', 'Vapis-sf'),
             ('feats',
              OrderedDict([('Aspect', 'Imp'),
                           ('Gender', 'Fem'),
                           ('Mood', 'Ind'),
                           ('Number', 'Sing'),
                           ('Tense', 'Past'),
                           ('VerbForm', 'Fin')])),
             ('head', 0),
             ('deprel', 'root'),
             ('deps', None),
             ('misc', OrderedDict([('Id', '0008')]))])

In [125]:
from enum import Enum
class Action_type(Enum):
    Left = 1
    Right = 2
    Reduse = 3
    Shift = 4
    
class oracle:
    def __init__(self, tree):
        self.rel=[]
        self.stack=[OrderedDict([('id', 0),('form','ROOT'),("head",0),("lemma","root")])]
        self.queue=tree
        self.step_processor={Action_type.Left:self.Left,
                             Action_type.Shift:self.Shift,
                             Action_type.Right:self.Right,
                             Action_type.Reduse:self.Reduse}
        
    def Shift(self):
        self.stack.append(self.queue.pop(0))
        
    def Left(self):
        self.rel.append((self.stack.pop()['id'],self.queue[0]['id']))
        
    def Right(self):
        self.rel.append((self.queue[0]['id'],self.stack[-1]['id']))
        self.stack.append(self.queue.pop(0))
        
    def Reduse(self):
        self.stack.pop()
        
    def do_steps(self, debug=False):
        history_step1=-1
        histiry_step2=-1
        self.features =[]
        self.labels = []
        self.dep = []
        while self.stack or self.queue:
            top_stack = self.stack[-1] if len(self.stack) > 0 else None
            first_queue =  self.queue[0] if len(self.queue) > 0 else None
            step_type=self.get_action(top_stack,first_queue)
            self.labels.append(step_type)
            if (step_type==Action_type.Left):
                self.dep.append(top_stack['deprel'])
            if (step_type==Action_type.Right):
                self.dep.append(first_queue['deprel'])
            else:
                self.dep.append('-')

            self.features.append(extract_features(self.stack, self.queue, self.rel,history_step1,histiry_step2))
            histiry_step2=history_step1
            history_step1=step_type.value
            if debug:
                self.info(step_type)
            self.step_processor[step_type]()
        return self.rel
    
    def get_action(self, stack,queue):
        if stack and not queue:
            return Action_type.Reduse
        if stack['head']==queue['id']:
             return Action_type.Left
        if stack['id']==queue['head']:
             return Action_type.Right
            
        if stack['id'] in  [i[0] for i in self.rel] and stack["id"] > queue["head"]:
             return Action_type.Reduse
        else:
            return Action_type.Shift
    
    def info(self, step_type):
        print('step: ', step_type)
        print('stack',  self.print_item(self.stack))
        print('queue',  self.print_item(self.queue))
        print('rel',  self.rel)
        print("====================================")

    def print_item(self, lst):
        return [i['form'] for i in lst]
    
    def get_relations(tree):
        return [(i['id'],i['head']) for i in tree]

# is static oracle do all transitions Well ?

In [199]:
correct_parsed_count=0
incorect_parsed_count = 0
incorect_tree_indexes=[]

for index,tree in enumerate(trees):
    o = oracle(tree.copy())
    parsed_relations = o.do_steps()
    gold_relations = oracle.get_relations(tree)
    correct_parsed_count+=len(set(gold_relations))
    incorect_parsed_count+= len(set(gold_relations)-set(parsed_relations)) +len(set(parsed_relations)-set(gold_relations)) 
    if set(parsed_relations)!=set(gold_relations):
        incorect_tree_indexes.append(index)

In [200]:
print ('trees count: ', len(trees))
print ('oracle_acc: ', correct_parsed_count/(incorect_parsed_count+correct_parsed_count))


trees count:  4513
oracle_acc:  0.9742738158560476


In [225]:
print(incorect_tree_indexes)

[4, 7, 8, 12, 13, 18, 21, 23, 26, 28, 32, 33, 34, 42, 43, 45, 46, 50, 53, 54, 66, 69, 110, 119, 127, 164, 169, 175, 177, 178, 179, 182, 194, 195, 200, 202, 203, 209, 220, 230, 244, 245, 248, 259, 270, 287, 288, 290, 302, 306, 313, 328, 344, 346, 353, 355, 364, 401, 403, 416, 423, 435, 436, 438, 441, 443, 453, 454, 461, 469, 480, 489, 491, 496, 497, 499, 500, 505, 507, 508, 509, 511, 516, 517, 522, 529, 530, 531, 537, 551, 554, 557, 579, 581, 634, 649, 665, 666, 679, 719, 789, 798, 819, 828, 835, 841, 863, 868, 869, 894, 903, 913, 963, 971, 974, 978, 982, 986, 989, 994, 1006, 1014, 1017, 1018, 1021, 1023, 1024, 1026, 1048, 1063, 1079, 1091, 1101, 1107, 1112, 1115, 1117, 1121, 1149, 1152, 1156, 1157, 1160, 1165, 1208, 1214, 1219, 1228, 1230, 1235, 1246, 1247, 1281, 1285, 1287, 1292, 1318, 1347, 1356, 1357, 1366, 1369, 1380, 1382, 1385, 1397, 1413, 1421, 1438, 1453, 1455, 1475, 1484, 1507, 1532, 1534, 1556, 1570, 1576, 1589, 1598, 1605, 1613, 1621, 1625, 1628, 1632, 1657, 1658, 1683, 1687

In [222]:
def analize_error(tree):
    o = oracle(tree.copy())
    parsed_relations = o.do_steps()
    gold_relations = oracle.get_relations(tree)
    print(set(gold_relations)- set(parsed_relations))
    print(set(parsed_relations)- set(gold_relations))
    print('-----------------------------------------')
    tree= tree
    for item in tree:
        print('{0}({1}) <- {2}({3})'.format(item['id'], item['form'], item['head'], tree[item['head'] - 1]["form"]
                             if item['head'] > 0 else "root"))
    
analize_error(trees[incorect_tree_indexes[0]])

{(35, 31), (44, 9)}
set()
-----------------------------------------
1(Як) <- 6(сюжетів)
2(і) <- 6(сюжетів)
3(багато) <- 6(сюжетів)
4(інших) <- 6(сюжетів)
5(її) <- 6(сюжетів)
6(сюжетів) <- 9(виявилася)
7(,) <- 6(сюжетів)
8(вона) <- 9(виявилася)
9(виявилася) <- 0(root)
10(візіонерським) <- 11(передбаченням)
11(передбаченням) <- 9(виявилася)
12(:) <- 23(почали)
13(за) <- 15(десятиліття)
14(якесь) <- 15(десятиліття)
15(десятиліття) <- 23(почали)
16(після) <- 17(написання)
17(написання) <- 15(десятиліття)
18(твору) <- 17(написання)
19(несамовиті) <- 20(ревнителі)
20(ревнителі) <- 23(почали)
21(нового) <- 22(ладу)
22(ладу) <- 20(ревнителі)
23(почали) <- 9(виявилася)
24(нищити) <- 23(почали)
25(культурні) <- 26(скарби)
26(скарби) <- 24(нищити)
27(,) <- 33(володіли)
28(до) <- 33(володіли)
29(того) <- 28(до)
30(ж) <- 28(до)
31(засобами) <- 33(володіли)
32(вони) <- 33(володіли)
33(володіли) <- 23(почали)
34(непорівнянно) <- 35(потужнішими)
35(потужнішими) <- 31(засобами)
36(,) <- 39(сектярі)
37(

In [214]:
#this is non-projective dependency tree, and . error

In [223]:
analize_error(trees[incorect_tree_indexes[1]])

{(8, 0), (1, 4)}
set()
-----------------------------------------
1(Мало) <- 4(років)
2(не) <- 1(Мало)
3(сто) <- 4(років)
4(років) <- 5(тому)
5(тому) <- 8(радив)
6(Микола) <- 8(радив)
7(Зеров) <- 6(Микола)
8(радив) <- 0(root)
9(не) <- 10(гаючись)
10(гаючись) <- 12(перевіяти)
11(«) <- 12(перевіяти)
12(перевіяти) <- 8(радив)
13(на) <- 14(току)
14(току) <- 12(перевіяти)
15(критики) <- 14(току)
16(») <- 12(перевіяти)
17(наші) <- 19(набутки)
18(мистецькі) <- 19(набутки)
19(набутки) <- 12(перевіяти)
20(,) <- 21(відсіяти)
21(відсіяти) <- 12(перевіяти)
22(полову) <- 21(відсіяти)
23(,) <- 24(виважити)
24(виважити) <- 12(перевіяти)
25(все) <- 26(найдобірніше)
26(найдобірніше) <- 24(виважити)
27(й) <- 28(означити)
28(означити) <- 12(перевіяти)
29(тяглість) <- 28(означити)
30(національної) <- 31(традиції)
31(традиції) <- 29(тяглість)
32(.) <- 8(радив)


In [ ]:
#need to fix

In [227]:
analize_error(trees[incorect_tree_indexes[2]])

{(7, 12), (10, 13), (6, 12), (32, 3), (12, 3)}
set()
-----------------------------------------
1(Лесю) <- 3(цензурували)
2(Українку) <- 1(Лесю)
3(цензурували) <- 0(root)
4(надзвичайно) <- 5(пильно)
5(пильно) <- 3(цензурували)
6(,) <- 12(мала)
7(адже) <- 12(мала)
8(в) <- 10(іконостасі)
9(радянському) <- 10(іконостасі)
10(іконостасі) <- 13(постати)
11(вона) <- 12(мала)
12(мала) <- 3(цензурували)
13(постати) <- 12(мала)
14(бездоганною) <- 15(марксисткою)
15(марксисткою) <- 13(постати)
16(,) <- 19(інтернаціоналісткою)
17(«) <- 19(інтернаціоналісткою)
18(пролетарською) <- 19(інтернаціоналісткою)
19(інтернаціоналісткою) <- 15(марксисткою)
20(») <- 19(інтернаціоналісткою)
21(,) <- 23(другом)
22(«) <- 23(другом)
23(другом) <- 15(марксисткою)
24(робітників) <- 23(другом)
25(») <- 23(другом)
26(,) <- 27(предтечею)
27(предтечею) <- 15(марксисткою)
28(соціалістичного) <- 29(реалізму)
29(реалізму) <- 27(предтечею)
30(et) <- 15(марксисткою)
31(cetera) <- 30(et)
32(.) <- 3(цензурували)


In [ ]:
non projective + dot problem

In [228]:
analize_error(trees[incorect_tree_indexes[3]])

{(25, 29), (3, 0), (26, 28), (38, 3), (1, 5), (29, 24), (34, 24), (30, 24)}
set()
-----------------------------------------
1(Найперш) <- 5(перегородити)
2(барикада) <- 3(мала)
3(мала) <- 0(root)
4(би) <- 3(мала)
5(перегородити) <- 3(мала)
6(затишну) <- 7(вітальню)
7(вітальню) <- 5(перегородити)
8(косачівського) <- 9(дому)
9(дому) <- 7(вітальню)
10(,) <- 24(стояла)
11(бо) <- 24(стояла)
12(,) <- 14(дізнаємося)
13(як) <- 14(дізнаємося)
14(дізнаємося) <- 24(стояла)
15(з) <- 17(пояснень)
16(академічних) <- 17(пояснень)
17(пояснень) <- 14(дізнаємося)
18(,) <- 14(дізнаємося)
19(Ольга) <- 24(стояла)
20(Драгоманова) <- 19(Ольга)
21(-) <- 22(Косач)
22(Косач) <- 20(Драгоманова)
23(«) <- 24(стояла)
24(стояла) <- 3(мала)
25(на) <- 29(позиціях)
26(ліберально) <- 28(буржуазних)
27(-) <- 26(ліберально)
28(буржуазних) <- 29(позиціях)
29(позиціях) <- 24(стояла)
30(») <- 24(стояла)
31(,) <- 34(донька)
32(а) <- 34(донька)
33(її) <- 34(донька)
34(донька) <- 24(стояла)
35(—) <- 37(марксистських)
36(на) <- 

(25, 29) need to fix, .dot problem

In [232]:
tree

[OrderedDict([('id', 1),
              ('form', 'Як'),
              ('lemma', 'як'),
              ('upostag', 'SCONJ'),
              ('xpostag', 'Css'),
              ('feats', None),
              ('head', 6),
              ('deprel', 'mark'),
              ('deps', None),
              ('misc', OrderedDict([('Id', '0020')]))]),
 OrderedDict([('id', 2),
              ('form', 'і'),
              ('lemma', 'і'),
              ('upostag', 'PART'),
              ('xpostag', 'Q'),
              ('feats', None),
              ('head', 6),
              ('deprel', 'discourse'),
              ('deps', None),
              ('misc', OrderedDict([('Id', '0021')]))]),
 OrderedDict([('id', 3),
              ('form', 'багато'),
              ('lemma', 'багато'),
              ('upostag', 'DET'),
              ('xpostag', 'Mlc-n'),
              ('feats',
               OrderedDict([('Case', 'Nom'),
                            ('NumType', 'Card'),
                            ('PronType', 'Ind')]

In [175]:
# generate features
def extract_features(stack, queue, rel, history_step1, histiry_step2):
    #print("stack",stack)
    stack1=stack[-1]['lemma'] if len(stack)>0 else ''
    stack2=stack[-2]['lemma'] if len(stack)>1 else ''
    stack3=stack[-3]['lemma'] if len(stack)>2 else ''
    queue1=queue[0]['lemma'] if len(queue)>0 else ''
    queue2=queue[1]['lemma'] if len(queue)>1 else ''
    queue3=queue[2]['lemma'] if len(queue)>2 else ''
    queue4=queue[3]['lemma'] if len(queue)>4 else ''
    
    features = dict()
    if len(stack) > 0:
        stack_top = stack[-1]
        s0_features = get_pymorphy_word_features(stack_top["form"],'s0_',stack2,queue1, True  )
    else:
        s0_features = get_pymorphy_word_features('there_is_no_word','s0_',stack2,queue1 , True )
    features = {**features, **s0_features}
    
    if len(stack) > 1:
        s1_features = get_pymorphy_word_features(stack[-2]["form"],'s1_',stack2,queue1 , True )
    else:
        s1_features = get_pymorphy_word_features('there_is_no_word','s1_',stack2,queue1, True )
    features = {**features, **s1_features}
    
        
    if queue:
        queue_top = queue[0]
        q0_features = get_pymorphy_word_features(queue_top["form"],'q0_',stack1,queue2, True )
    else: 
        q0_features = get_pymorphy_word_features('there_is_no_word','q0_',stack1,queue2, True )
    features = {**features, **q0_features}
    
    if len(queue)>1:
        queue1 = queue[1]
        q1_features = get_pymorphy_word_features(queue1["form"],'q1_',stack1,queue3)
    else: 
        q1_features = get_pymorphy_word_features('there_is_no_word','q1_',stack1,queue3)
    features = {**features, **q1_features}
    
    if len(queue)>1:
        queue1 = queue[1]
        q2_features = get_pymorphy_word_features(queue1["form"],'q2_',stack1,queue4)
    else: 
        q2_features = get_pymorphy_word_features('there_is_no_word','q2_',stack1,queue4)
    features = {**features, **q2_features}
    
    features['history_step1']=history_step1
    features['histiry_step2']=histiry_step2


    features['stack_len']=len(stack)
    features['queue_len']=len(queue)
    features['rel_len']=len(rel)
    features['parrent_of']= sum([int(i[1]==stack_top['id']) for i in rel])
    if stack and queue:
        features["distance"] = queue[0]["id"] - stack[-1]["id"]
        features["word_index"] = queue[0]["id"]
    else:
        features["distance"]=-1
        features["word_index"]=-1
    return features

import pymorphy2
morph = pymorphy2.MorphAnalyzer(lang='uk')

def get_pymorphy_word_features(word, prefix, left, right, add_embeding=False):
    word_info = morph.parse(word)[0]
    try:
        left_tag = morph.parse(left)[0].tag.POS
    except:
        left_tag=''
    try:
        right_tag = morph.parse(right)[0].tag.POS
    except:
        right_tag=''

    features = dict()
    if add_embeding:
        try:
            word_vector=model.get_vector(word_info.normal_form.lower())
        except:
            word_vector=[0]*300
        for i in range(0,300):
            features[prefix+'vector+'+str(i)]=word_vector[i]
    features[prefix+'POS']=word_info.tag.POS
    features[prefix+'animacy']=word_info.tag.animacy
    features[prefix+'aspect']=word_info.tag.aspect
    features[prefix+'case']=word_info.tag.case
    features[prefix+'number']=word_info.tag.number
    features[prefix+'number']=word_info.tag.number
    features[prefix+'person']=word_info.tag.person
    features[prefix+'tense']=word_info.tag.tense
    features[prefix+'transitivity']=word_info.tag.transitivity
    features[prefix+'left_item']= '{0}_{1}'.format(left_tag, word_info.tag.POS)
    features[prefix+'item_right']= '{0}_{1}'.format(word_info.tag.POS,right_tag)
    features[prefix+'left_right']= '{0}_{1}'.format(left_tag, right_tag)

    return features

In [292]:
model.init_sims(replace=True)

# collect  features

In [95]:
label_encode_columns = ['POS','animacy','aspect','case','number','person','tense','item','left_right','pref','suf']

In [96]:
from sklearn import preprocessing
label_encoders={}

In [103]:
label_encoders

{'POS': LabelEncoder(),
 'animacy': LabelEncoder(),
 'aspect': LabelEncoder(),
 'case': LabelEncoder(),
 'item': LabelEncoder(),
 'left_right': LabelEncoder(),
 'number': LabelEncoder(),
 'person': LabelEncoder(),
 'tense': LabelEncoder()}

In [97]:
import pandas as pd

In [174]:
def get_vectors(trees):
    from tqdm import tqdm
    features =[]
    labels = []
    dep_labels = []


    for index in tqdm(range(0,len(trees))):
        tree=trees[index]
        o = oracle(tree.copy())
        parsed_relations = o.do_steps()
        features=features + o.features
        labels=labels + o.labels
        dep_labels=dep_labels+o.dep
    
    features_df = pd.DataFrame(features)
    features_df.shape
    features_df.fillna(0, inplace=True)
    #print(features_df)
    for sub in label_encode_columns:
        for col in features_df.columns:
            if sub in col:
                if sub not in label_encoders:
                    #print('train encoder', sub)
                    label_encoders[sub]= preprocessing.LabelEncoder()
                    val = []
                    for col_serch in features_df.columns:
                        if sub in col_serch:
                            features_df[col_serch]=features_df[col_serch].astype(str)
                            val.append(features_df[col_serch].values)
                    label_encoders[sub].fit(np.concatenate(val))
                #print(col)
                features_df[col]=features_df[col].astype(str)
                safe_transform(label_encoders[sub],features_df, col)
    return features_df, labels, dep_labels

In [122]:
def safe_transform(labelEncoder, df,column_name):
    dic =dict(zip(labelEncoder.classes_, labelEncoder.transform(labelEncoder.classes_)))
    df[column_name]=df[column_name].map(dic).fillna(-1).astype(int)

In [127]:
with open('../UD_Ukrainian-IU/uk_iu-ud-train.conllu') as f:
    c= f.read()
    trees= parse(c)

X_train, y_train, y_train_dep =get_vectors(trees)


100%|██████████| 4513/4513 [06:23<00:00, 11.76it/s]


q0_POS
q1_POS
q2_POS
s0_POS
s1_POS
q0_animacy
q1_animacy
q2_animacy
s0_animacy
s1_animacy
q0_aspect
q1_aspect
q2_aspect
s0_aspect
s1_aspect
q0_case
q1_case
q2_case
s0_case
s1_case
q0_number
q1_number
q2_number
s0_number
s1_number
q0_person
q1_person
q2_person
s0_person
s1_person
q0_tense
q1_tense
q2_tense
s0_tense
s1_tense
q0_item_right
q0_left_item
q1_item_right
q1_left_item
q2_item_right
q2_left_item
s0_item_right
s0_left_item
s1_item_right
s1_left_item
q0_left_right
q1_left_right
q2_left_right
s0_left_right
s1_left_right


In [128]:
1+1

2

In [129]:
y_train = [i.value for i in y_train]

# get test

In [130]:
with open('../UD_Ukrainian-IU/uk_iu-ud-test.conllu') as f:
    c= f.read()
    trees= parse(c)

X_test, y_test, y_test_dep =get_vectors(trees)
y_test = [i.value for i in y_test] 

100%|██████████| 783/783 [01:03<00:00, 12.34it/s]


q0_POS
q1_POS
q2_POS
s0_POS
s1_POS
q0_animacy
q1_animacy
q2_animacy
s0_animacy
s1_animacy
q0_aspect
q1_aspect
q2_aspect
s0_aspect
s1_aspect
q0_case
q1_case
q2_case
s0_case
s1_case
q0_number
q1_number
q2_number
s0_number
s1_number
q0_person
q1_person
q2_person
s0_person
s1_person
q0_tense
q1_tense
q2_tense
s0_tense
s1_tense
q0_item_right
q0_left_item
q1_item_right
q1_left_item
q2_item_right
q2_left_item
s0_item_right
s0_left_item
s1_item_right
s1_left_item
q0_left_right
q1_left_right
q2_left_right
s0_left_right
s1_left_right


In [131]:
import gc
gc.collect()

45363

In [132]:
1+1

2

In [133]:
test_trees=trees

# build model

In [138]:
from sklearn.ensemble import RandomForestClassifier

In [152]:
clf = RandomForestClassifier(max_depth=19, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=19, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [153]:
predicted_test = clf.predict(X_test)

In [154]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted_test, target_names=['Left','Right','Reduse','Shift']))

             precision    recall  f1-score   support

       Left       0.81      0.90      0.86      7352
      Right       0.76      0.77      0.76      7182
     Reduse       0.82      0.76      0.79      8370
      Shift       0.87      0.84      0.86      7757

avg / total       0.82      0.82      0.82     30661



# feature importance

In [142]:
 X_test.columns[clf.feature_importances_.argsort()[::-1][:40]]

Index(['distance', 'queue_len', 'q0_item_right', 'q1_left_item', 'q2_POS',
       'word_index', 'history_step1', 'q0_vector+173', 'q0_vector+197',
       'q0_left_item', 'q2_left_item', 'q0_vector+100', 'parrent_of',
       'q0_case', 's0_item_right', 'q0_POS', 'q2_left_right', 'q2_case',
       'q0_animacy', 'q0_aspect', 'q0_tense', 'q1_case', 'q0_vector+13',
       'q0_vector+126', 'q1_POS', 'q0_vector+230', 'q1_item_right',
       'q0_vector+41', 'q0_vector+293', 'q0_vector+175', 'q0_vector+32',
       'histiry_step2', 'q2_item_right', 'q0_vector+149', 'q0_vector+240',
       'q1_left_right', 'q0_left_right', 's0_animacy', 'q0_vector+250',
       's0_aspect'],
      dtype='object')

# generate featurs

In [ ]:
next steps:
1 fix bugs in oracl transitions
4 запустити лайт gbm
5 динамічний оракул


# Calculate the unlabeled attachment score

In [177]:
def get_df_features(features): 
    features_df = pd.DataFrame(features)
    features_df.shape
    features_df.fillna(0, inplace=True)
    #print(features_df)
    for sub in label_encode_columns:
        for col in features_df.columns:
            if sub in col:
                if sub not in label_encoders:
                    #print('train encoder', sub)
                    label_encoders[sub]= preprocessing.LabelEncoder()
                    val = []
                    for col_serch in features_df.columns:
                        if sub in col_serch:
                            features_df[col_serch]=features_df[col_serch].astype(str)
                            val.append(features_df[col_serch].values)
                    label_encoders[sub].fit(np.concatenate(val))
                #print(col)
                features_df[col]=features_df[col].astype(str)
                safe_transform(label_encoders[sub],features_df, col)
    return features_df

In [170]:
def dep_parse(sentence, model):
    stack, queue, relations = [OrderedDict([('id', 0),('form','ROOT'),("head",0),("lemma","root")])], sentence[:], []
    history_step1=-1
    history_step2=-1
    while queue or stack:
        if stack and not queue:
            stack.pop()
        else:
            features = extract_features(stack, queue, relations,history_step1, history_step2)
            df = get_df_features([features])
            action = model.predict(df.values)[0]
            # actual parsing
            if action == 4: #:Action_type.Shift:
                stack.append(queue.pop(0))
            elif action == 3: #:Action_type.Reduse:
                stack.pop()
            elif action == 1: #Action_type.Left:
                relations.append((stack[-1]["id"], queue[0]["id"]))
                stack.pop()
            elif action == 2: #Action_type.Right:
                relations.append((queue[0]["id"], stack[-1]["id"]))
                stack.append(queue.pop(0))
            else:
                print("Unknown action.", action)
            histiry_step2=history_step1
            history_step1=action
    return sorted(relations)

In [182]:
print([node["form"] for node in test_trees[1]])
print(dep_parse(test_trees[1], clf))
print([(node["id"], node["head"]) for node in test_trees[1]])

['Продавши', 'свій', 'шедевр', 'Меценатові', ',', 'еллінський', 'скульптор', 'споневажив', 'саме', 'мистецтво', ':', 'Ти', 'не', 'продався', ',', '—', 'гірше', '!']
[(1, 0), (2, 3), (3, 1), (4, 3), (5, 8), (6, 7), (7, 8), (8, 1), (9, 10), (10, 8), (11, 14), (12, 14), (13, 14), (14, 1), (15, 1), (18, 1)]
[(1, 8), (2, 3), (3, 1), (4, 1), (5, 1), (6, 7), (7, 8), (8, 0), (9, 10), (10, 8), (11, 14), (12, 14), (13, 14), (14, 8), (15, 17), (16, 17), (17, 14), (18, 14)]


In [180]:
gc.collect()

613

In [181]:
from tqdm import tqdm 
total, tp = 0, 0
for index in tqdm(range(0,len(test_trees))):
    tree=test_trees[index]
    golden = [(node["id"], node["head"]) for node in tree]
    predicted = dep_parse(tree, clf)
    total += len(tree)
    tp += len(set(golden).intersection(set(predicted)))

print("Total:", total)
print("Correctly defined:", tp)
print("UAS:", round(tp/total, 2))


  0%|          | 2/783 [00:06<39:41,  3.05s/it]
Exception in thread Thread-25:
Traceback (most recent call last):
  File "/Users/grycshuknazar/anaconda2/envs/env_36/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/grycshuknazar/anaconda2/envs/env_36/lib/python3.6/site-packages/tqdm/_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/grycshuknazar/anaconda2/envs/env_36/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 783/783 [48:16<00:00,  3.70s/it] 

Total: 14939
Correctly defined: 9950
UAS: 0.67
